## Pivot
- 데이터 프레임 컬럼데이터에서 특정 컬럼의 데이터를 index, columns, values를 선택하여 프레임을 만드는 방법입니다.
- df.pivot(index, columns, values)

In [1]:
import pandas as pd
import numpy as np

In [7]:
#타이타닉데이터
#survived - 0:no, 1:yes
titanic = pd.read_csv('train.csv')
titanic.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


### 성별에 따른 등급부터 만들자
    등급    1     2     3
     -------------------------------
성별  female 000   000   000   
     male   000   000   000

In [10]:
#중복데이터는 groupby로 중복을 없애주고 pivot을 사용해야한다.
#groupby

titanic_df1 = titanic.groupby(["Sex", "Pclass"]).size().reset_index(name="Counts")
titanic_df1

,Sex,Pclass,Counts
0,female,1,94
1,female,2,76
2,female,3,144
3,male,1,122
4,male,2,108
5,male,3,347


In [11]:
titanic_df1 = titanic_df1.pivot("Sex","Pclass","Counts")
titanic_df1

Pclass,1,2,3
Sex,,,
female,94,76,144
male,122,108,347


# 이번엔 생존데이터를 만들어보자
### 성별에 따른 생존자수를 나타내는 데이터프레임

생존  0   1
성별
-------------------
female 000 000
male 000 000

In [16]:
titanic_df2 = titanic.groupby(["Sex","Survived"]).size().reset_index(name = "Counts")
titanic_df2

,Sex,Survived,Counts
0,female,0,81
1,female,1,233
2,male,0,468
3,male,1,109


In [17]:
titanic_df2 = titanic_df2.pivot("Sex","Survived","Counts")
titanic_df2

Survived,0,1
Sex,,
female,81,233
male,468,109


In [18]:
#객실등급에 따른 생존자수

titanic_df3 = titanic.groupby(["Pclass","Survived"]).size().reset_index(name="Counts")
titanic_df3

,Pclass,Survived,Counts
0,1,0,80
1,1,1,136
2,2,0,97
3,2,1,87
4,3,0,372
5,3,1,119


In [19]:
titanic_df3 = titanic_df3.pivot("Pclass","Survived","Counts")
titanic_df3

Survived,0,1
Pclass,,
1,80,136
2,97,87
3,372,119


## pivot table

- 위에서는 먼저 groupby로 정리하고 pivot했지만, 이것은 한방에 해치운다.
- pivot_table(values, index, columns, aggfunc)
- fill_value : NaN 데이터를 설정한 데이터로 치환해주는 파라미터

- dropna(True) : NaN 데이터컬럼을 놔둘지 제거할지 결정한다.

In [22]:
titanic["Counts"] = 1
titanic.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Counts
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S,1
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S,1
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C,1
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q,1


In [39]:
a = titanic.pivot_table("Counts",["Sex"], ["Pclass"], aggfunc=np.sum)
a

Pclass,1,2,3
Sex,,,
female,94,76,144
male,122,108,347


In [40]:
#pivot_table로 성별에 따른 생존자 수를 뽑아보자
A = titanic.pivot_table("Counts",["Sex"],["Survived"], aggfunc=np.sum)
A

Survived,0,1
Sex,,
female,81,233
male,468,109


In [41]:
#pivot_table로 좌석등급에 따른 생존자 수도 뽑아보자
B = titanic.pivot_table("Counts",["Pclass"],["Survived"], aggfunc=np.sum)
B

Survived,0,1
Pclass,,
1,80,136
2,97,87
3,372,119


In [42]:
df = titanic.pivot_table("Counts", ["Sex","Pclass"], ["Survived"], aggfunc=np.sum)
df

Survived         0   1
Sex    Pclass         
female 1         3  91
       2         6  70
       3        72  72
male   1        77  45
       2        91  17
       3       300  47

---

In [43]:
df = titanic.pivot_table("Counts",["Survived"],["Sex"],aggfunc=np.sum)
df

Sex,female,male
Survived,,
0,81,468
1,233,109


In [55]:
df["total"] = df["female"] + df["male"]
df

Sex,female,male,total
Survived,,,
0,81,468,549
1,233,109,342


In [56]:
df.loc["total"] = df.loc[0] + df.loc[1]
df


Sex,female,male,total
Survived,,,
0,81,468,549
1,233,109,342
total,314,577,891


In [57]:
#total데이터행을 drop으로 삭제해보자
df.drop("total", inplace=True) #axis =0 일때는 행을 삭제한다
df


Sex,female,male,total
Survived,,,
0,81,468,549
1,233,109,342


In [58]:
#이번에는 total데이터 열을 drop으로 삭제해보자
df.drop("total", inplace=True, axis = 1) #axis =0 일때는 행을 삭제한다
df

Sex,female,male
Survived,,
0,81,468
1,233,109


---


In [61]:
#fill_value (NaN 데이터 대신 넣어줄 값을 정하는 설정)

df = titanic.pivot_table("Counts",["Survived"],["Parch","Pclass"], aggfunc=np.sum, fill_value=0)
df

Parch      0            1           2          3     4     5  6
Pclass     1   2    3   1   2   3   1   2   3  2  3  1  3  3  3
Survived                                                       
0         64  86  295  10   8  35   5   3  32  0  2  1  3  4  1
1         99  48   86  21  24  20  16  13  11  2  1  0  0  1  0

In [63]:
#dropna (NaN값으로만 이루어진 행과 열을 그냥 제거할지 여부 결정 = False면 남긴다.)

df = titanic.pivot_table("Counts",["Survived"],["Parch","Pclass"], aggfunc=np.sum, dropna=False)
df

Parch        0                  1                 2               3 ...        \
Pclass       1     2      3     1     2     3     1     2     3   1 ...     3   
Survived                                                            ...         
0         64.0  86.0  295.0  10.0   8.0  35.0   5.0   3.0  32.0 NaN ...   2.0   
1         99.0  48.0   86.0  21.0  24.0  20.0  16.0  13.0  11.0 NaN ...   1.0   

Parch       4            5            6           
Pclass      1   2    3   1   2    3   1   2    3  
Survived                                          
0         1.0 NaN  3.0 NaN NaN  4.0 NaN NaN  1.0  
1         NaN NaN  NaN NaN NaN  1.0 NaN NaN  NaN  

[2 rows x 21 columns]